In [ ]:
!git clone "https://github.com/domiurg/ML_class_kaggle"

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
# from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
# from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os


In [ ]:
training_set_path = 'ML_class_kaggle/train_kaggle'
val_set_path = 'ML_class_kaggle/val_kaggle'

class DataGenerator(object):
    def __init__(self, train_path='', val_path='', preprocess=None, img_size=(224, 224), b_size=4):
        self.train_path = train_path
        self.val_path = val_path
        self.preprocess = preprocess
        self.img_size = img_size
        self.b_size = b_size
        self.train_datagen = ImageDataGenerator(preprocessing_function=self.preprocess,
                                                rotation_range=40,
                                                width_shift_range=0.2,
                                                height_shift_range=0.2,
                                                shear_range=0.2,
                                                zoom_range=0.2,
                                                channel_shift_range=10,
                                                horizontal_flip=True,
                                                fill_mode='nearest')
        self.val_datagen = valid_datagen = ImageDataGenerator(preprocessing_function=self.preprocess)

    def get_batches(self):
        train_batches = self.train_datagen.flow_from_directory(self.train_path,
                                                               target_size=self.img_size,
                                                               interpolation='bicubic',
                                                               class_mode='categorical',
                                                               shuffle=True,
                                                               batch_size=self.b_size)
        val_bathces = self.val_datagen.flow_from_directory(self.val_path,
                                                           target_size=self.img_size,
                                                           interpolation='bicubic',
                                                           class_mode='categorical',
                                                           shuffle=False,
                                                           batch_size=self.b_size)
        return train_batches, val_bathces

In [ ]:
IMAGE_SIZE = (224, 224)
NUM_CLASSES = 5
BATCH_SIZE = 4  # try reducing batch size or freeze more layers if your GPU runs out of memory
NUM_EPOCHS = 1500
WEIGHTS_FINAL = 'model-resnet50v2-final.h5'

datagen = DataGenerator(training_set_path, val_set_path, preprocess_input, IMAGE_SIZE, BATCH_SIZE)
train_batches, val_batches = datagen.get_batches()

# show class indices
print('****************')
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))
print('****************')

In [ ]:
net = ResNet50V2(include_top=False, weights='imagenet', input_tensor=None,
                  input_shape=(*IMAGE_SIZE, 3))
x = net.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)
net_final = Model(inputs=net.input, outputs=output_layer)

for layer in net.layers[:len(net.layers) - 12]:
    layer.trainable = False
net_final.compile(optimizer=Adam(lr=0.0001),
                  loss='categorical_crossentropy', metrics=['accuracy'])

print(net_final.summary())
print(len(net.layers))

In [ ]:
# Define Callbacks
filepath = "res50v2-best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1,
                             save_best_only=True, mode='max')
early_stop = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=150)
csv_history = 'res50v2-history.csv'
csv_logger = CSVLogger(csv_history, append=False)
callbacks = [checkpoint, early_stop, csv_logger]

In [ ]:
# train the model
H = net_final.fit_generator(train_batches,
                            steps_per_epoch=train_batches.samples // BATCH_SIZE,
                            validation_data=val_batches,
                            validation_steps=val_batches.samples // BATCH_SIZE,
                            epochs=NUM_EPOCHS,
                            callbacks=callbacks)

net_final.save(WEIGHTS_FINAL)